# Notebook: Filter Sentences from Reviews to create Dataset for Annotation

## Packages

In [1]:
from bs4 import BeautifulSoup
from langdetect import detect
import pandas as pd
import json
import nltk
import uuid
from nltk.tokenize import sent_tokenize
import re

## Settings

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nils_hellwig/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Constants

In [3]:
REVIEWS_PATH = "reviews_dataset/reviews.csv"
RANDOM_STATE = 43

## Code

### Load Dataset

In [4]:
df_reviews = pd.read_csv(REVIEWS_PATH)

In [5]:
len(df_reviews)

3212

In [6]:
# Höchstes Datum
max_date = df_reviews['date'].max()

# Niedrigstes Datum
min_date = df_reviews['date'].min()

print("Höchstes Datum:", max_date)
print("Niedrigstes Datum:", min_date)


Höchstes Datum: 2023-10-15
Niedrigstes Datum: 2022-10-15


### Filter Sentences

In [7]:
df_reviews_sentences = pd.DataFrame(columns=list(df_reviews.columns) + ['sentence_idx', "id"])

for idx, row in df_reviews.iterrows():
    sentences = sent_tokenize(row['text'], language="german")
    sentence_index = 0
    for sentence in sentences:
        if len(sentence) > 1:
            new_row = row.copy() 
            new_row['text'] = sentence 
            new_row['id'] = str(uuid.uuid4())
            new_row['sentence_idx'] = sentence_index
            df_reviews_sentences.loc[len(df_reviews_sentences)] = new_row
            sentence_index += 1

In [8]:
df_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id
0,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Bayrisches Bier mitten in LOC.Das kann man im ...,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,0,442da429-16fa-41d4-8909-fe0dd98f6158
1,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,"Das Lokal ist immer voll, eine Reservierung is...",5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,1,fece6f61-1651-4503-9d89-3cd29309de1c
2,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Innen ist es gemütlich aber auch laut.,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,2,88ab5e50-d7de-40a4-b88a-ab98789e351e
3,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Ein Teil des Personals versucht sich südbayeri...,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,3,688f5141-cd5c-4f0d-a8b3-9fc5dc4dbf6a
4,1,921129385,2005104,0,Denn Sie wissen nicht was Sie tun!!!,2023-10-13,Maximilian-62,"Freiburg, Deutschland",Bayrisch Essen und Gastfreundschaft geht anders.,2.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisch Essen und Gastfreundschaft geht ander...,0,f7ffdca5-6e76-40e3-b38d-a54e6c8a67b5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13421,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Haxe wo man noch nicht mal ein Messer braucht.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,1,6e1cc113-77e0-4f33-a706-4313ab2a723a
13422,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Und ich weiß dass ich niemals irgendwo anders ...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,2,1c4df80f-031e-4641-bd09-1c5165b776eb
13423,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Super nettes aufmerksames Personal.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,3,24574962-8c2d-4c15-9954-a816e5fe7555
13424,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Chef kommt auch selber an den Tisch und ist se...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,4,f3bdb7ef-6a76-4d8b-8206-f0999a1b129d


### Filter Non-German Sentences

Even if the language code = "de", we have observed that some sentences are in other languagesthan german. These are excluded as well with the help of Google's language-detection.

In [9]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'Unknown'

df_reviews_sentences['detected_language'] = df_reviews_sentences['text'].apply(detect_language)
df_reviews_sentences[df_reviews_sentences['detected_language'] != 'de']

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
21,3,918788541,2005104,0,Nicht zu empfehlen! Zero Dienstleistungsverstä...,2023-09-28,cariocadagema74,"Berlin, Deutschland",Kein Cent!,1.0,berlin,Augustiner am Gendarmenmarkt,de,Dieser Laden macht den Ruhm Berlins als Servic...,10,d2c66afa-98a2-4d71-815e-7cb73869c366,nl
54,9,890917149,2005104,0,schlecht verwaltetes Chaos,2023-05-18,Harald F,"Berlin, Deutschland",Es war viel los.,1.0,berlin,Augustiner am Gendarmenmarkt,de,ich hätte gerne etwas zum Essen gesagt. Aber e...,3,aac30680-1a69-42f5-9e56-b2b2c8a9c3b9,es
103,18,866452621,2005104,1,Mind this touri-gap,2022-10-29,452stephand,"Nürnberg, Deutschland",Fazit: mind this touri-gap,1.0,berlin,Augustiner am Gendarmenmarkt,de,Wunderschöne Location mit mangelhafter Leitung...,1,b3096715-40e1-41c6-9f78-94b9278793dc,en
111,21,883653345,947082,0,Tolle Burger!,2023-03-26,dittelbrunn,"Lübeck, Deutschland",Ketchup.,5.0,berlin,Burgermeister,de,Wir waren in der Filiale am S-Bahnhof Eberswal...,2,419f66d8-44f4-411b-a2e8-de91c32163eb,en
115,22,883285244,947082,0,Ganz gut,2023-03-23,Mawenabert,NaN,Ansonsten alles gut,4.0,berlin,Burgermeister,de,Hatte einen Hausmeister Burger mit Pommes Baco...,2,bd05a9cf-a32e-443a-9192-01c6bff8fce5,fr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13318,3180,873342513,13144674,5,Wie immer super,2022-12-30,nelala79,NaN,Service super.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Salat mit Genüsenuggets gegessen. Wie man es h...,2,95d67312-8c84-4b7e-afd8-aa782b00746a,en
13319,3180,873342513,13144674,5,Wie immer super,2022-12-30,nelala79,NaN,Essen super.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Salat mit Genüsenuggets gegessen. Wie man es h...,3,740a4712-4ff2-4ba7-8251-c598e3d6e03f,ca
13322,3180,873342513,13144674,5,Wie immer super,2022-12-30,nelala79,NaN,Klar!,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Salat mit Genüsenuggets gegessen. Wie man es h...,6,7413a5e1-7487-421b-ae0c-fb257e60e85c,tr
13356,3195,866578862,13144674,6,Leckeres Essen in Mönchengladbach,2022-10-30,marcelbF2818DO,NaN,Ein tolles Restaurant!,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Ein tolles Restaurant! Ambiente und Empfang su...,0,76eb921d-9316-4f35-8e05-c7366a1699ab,fr


In [10]:
df_reviews_sentences = df_reviews_sentences[df_reviews_sentences['detected_language'] == 'de']

In [11]:
df_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
0,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Bayrisches Bier mitten in LOC.Das kann man im ...,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,0,442da429-16fa-41d4-8909-fe0dd98f6158,de
1,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,"Das Lokal ist immer voll, eine Reservierung is...",5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,1,fece6f61-1651-4503-9d89-3cd29309de1c,de
2,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Innen ist es gemütlich aber auch laut.,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,2,88ab5e50-d7de-40a4-b88a-ab98789e351e,de
3,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Ein Teil des Personals versucht sich südbayeri...,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,3,688f5141-cd5c-4f0d-a8b3-9fc5dc4dbf6a,de
4,1,921129385,2005104,0,Denn Sie wissen nicht was Sie tun!!!,2023-10-13,Maximilian-62,"Freiburg, Deutschland",Bayrisch Essen und Gastfreundschaft geht anders.,2.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisch Essen und Gastfreundschaft geht ander...,0,f7ffdca5-6e76-40e3-b38d-a54e6c8a67b5,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13421,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Haxe wo man noch nicht mal ein Messer braucht.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,1,6e1cc113-77e0-4f33-a706-4313ab2a723a,de
13422,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Und ich weiß dass ich niemals irgendwo anders ...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,2,1c4df80f-031e-4641-bd09-1c5165b776eb,de
13423,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Super nettes aufmerksames Personal.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,3,24574962-8c2d-4c15-9954-a816e5fe7555,de
13424,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Chef kommt auch selber an den Tisch und ist se...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,4,f3bdb7ef-6a76-4d8b-8206-f0999a1b129d,de


### Balancing

In [12]:
df_balanced_reviews_sentences = df_reviews_sentences.groupby(['rating'], group_keys=False).apply(lambda x: x.sample(800, replace=False, random_state=RANDOM_STATE))
df_balanced_reviews_sentences.reset_index(drop=True, inplace=True)

### Randomisierung

In [13]:
df_balanced_reviews_sentences = df_balanced_reviews_sentences.sample(frac=1, random_state=RANDOM_STATE)  # 'frac=1' mischt den gesamten DataFrame
df_balanced_reviews_sentences.reset_index(drop=True, inplace=True)

### Check If Sentences are Unique

In [14]:
df_balanced_reviews_sentences.groupby(["review_id", "sentence_idx"]).size().reset_index(name="count")

,review_id,sentence_idx,count
0,864598719,0,1
1,864598719,2,1
2,864598719,3,1
3,864598719,5,1
4,864603326,0,1
...,...,...,...
3995,921404419,7,1
3996,921407021,0,1
3997,921412151,0,1
3998,921446681,0,1


### Alternate examples based on rating

In [15]:
df_balanced_reviews_sentences["rating"].value_counts()

rating
1.0    800
3.0    800
5.0    800
2.0    800
4.0    800
Name: count, dtype: int64

In [16]:
dfs = [df_balanced_reviews_sentences[df_balanced_reviews_sentences["rating"] == i] for i in range(1, 6)]
df_alternating = pd.DataFrame(columns=df_balanced_reviews_sentences.columns)

max_rows = max(len(df) for df in dfs)
for i in range(max_rows):
    for k in range(5):
        if i < len(dfs[k]):
            df_alternating = pd.concat([df_alternating, dfs[k].iloc[[i]]])

df_balanced_reviews_sentences = df_alternating.reset_index(drop=True)

In [17]:
df_balanced_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
0,901,868199195,1190751,13,Unfassbar schlechter Service. Macht einen Boge...,2022-11-12,829ingos,"Frankfurt am Main, Deutschland","Obwohl wir reserviert hatten, war unser Platz ...",1.0,frankfurt,Trattoria i Siciliani,de,Unfassbar unfreundlicher Service. Obwohl wir r...,1,78981b3d-544e-46a1-b9a7-91ff4701d026,de
1,1520,908437582,743219,0,Enttäuschend,2023-08-07,326jensn,NaN,"Ich habe dort keine schöne Erfahrung gemacht, ...",2.0,leipzig,Cafe Riquet,de,"Ich habe dort keine schöne Erfahrung gemacht, ...",0,45c0c215-bf53-473f-bf9d-b3d9f3a10ef7,de
2,644,920079618,718846,0,Mittelklasse,2023-10-06,oesehh,"Hamburg, Deutschland",Das Essen war dann für Münchner Brauhäuser ehe...,3.0,münchen,Schneider Bräuhaus,de,Bier kann man gut trinken und der Service war ...,1,28a893a8-0944-42c0-a80c-a40a2d17e7f1,de
3,1654,895902732,1343851,0,Solide und ehrlich bei akzeptablen Preisen,2023-06-19,volkers542,"Sindelfingen, Deutschland",Das Ambiente ist natürlich besonders.,4.0,bremen,Bremer Ratskeller,de,Vor der geführten Besichtigung des Rathauses h...,6,d6c3730a-097d-465d-b48c-d9e0b35e5d07,de
4,397,886089135,10322027,14,Super Ambiente und noch besserer Service!,2023-04-13,Ruksson,"Celle, Deutschland","Ich hatte das Vergnügen, im LOC zu speisen und...",5.0,hamburg,Ola Lisboa,de,"Ich hatte das Vergnügen, im Ola Lisboa zu spei...",0,86029d68-a742-4708-906c-22b09557e709,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,921,883130438,6847690,0,Massivst überbewertet,2023-03-22,Comillano,"Frankfurt am Main, Deutschland",Totale Abzocke und massivst überbewertet.,1.0,frankfurt,Thai & Turf - Steakhouse,de,Totale Abzocke und massivst überbewertet. Das ...,0,f6a8f9aa-3ec0-4090-86c4-4629d06018b3,de
3996,603,874144809,947503,2,"Hier geht man nur rein, wenn man woanders kein...",2023-01-06,Bernhard R,Erding,Schönes bairisches Wirtshaus mit katastrophale...,2.0,münchen,Augustiner Stammhaus,de,Schönes bairisches Wirtshaus mit katastrophale...,0,3933eb71-5b09-4edc-aa96-5323ae42370a,de
3997,2009,865055047,967507,1,Mehr Schein als Sein!,2022-10-18,reinhold0913,NaN,Die Sauce und das Gemüse waren in Ordnung.,3.0,dresden,Restaurant & Grand Café Coselpalais,de,Wir waren zum Abendessen im Coselpalais. Ein s...,6,b12cb643-9ef0-494b-adf8-713833595562,de
3998,1654,895902732,1343851,0,Solide und ehrlich bei akzeptablen Preisen,2023-06-19,volkers542,"Sindelfingen, Deutschland",Jahrhundertealte Weinfassböden mit tollen Schn...,4.0,bremen,Bremer Ratskeller,de,Vor der geführten Besichtigung des Rathauses h...,7,08638290-6f50-4440-b386-aae8b8a9601c,de


### Store as .csv 

In [18]:
df_reviews_sentences.to_csv("reviews_dataset/reviews_sentences.csv")

In [19]:
df_balanced_reviews_sentences.to_csv("reviews_dataset/balanced_reviews_sentences.csv")

In [20]:
df_balanced_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
0,901,868199195,1190751,13,Unfassbar schlechter Service. Macht einen Boge...,2022-11-12,829ingos,"Frankfurt am Main, Deutschland","Obwohl wir reserviert hatten, war unser Platz ...",1.0,frankfurt,Trattoria i Siciliani,de,Unfassbar unfreundlicher Service. Obwohl wir r...,1,78981b3d-544e-46a1-b9a7-91ff4701d026,de
1,1520,908437582,743219,0,Enttäuschend,2023-08-07,326jensn,NaN,"Ich habe dort keine schöne Erfahrung gemacht, ...",2.0,leipzig,Cafe Riquet,de,"Ich habe dort keine schöne Erfahrung gemacht, ...",0,45c0c215-bf53-473f-bf9d-b3d9f3a10ef7,de
2,644,920079618,718846,0,Mittelklasse,2023-10-06,oesehh,"Hamburg, Deutschland",Das Essen war dann für Münchner Brauhäuser ehe...,3.0,münchen,Schneider Bräuhaus,de,Bier kann man gut trinken und der Service war ...,1,28a893a8-0944-42c0-a80c-a40a2d17e7f1,de
3,1654,895902732,1343851,0,Solide und ehrlich bei akzeptablen Preisen,2023-06-19,volkers542,"Sindelfingen, Deutschland",Das Ambiente ist natürlich besonders.,4.0,bremen,Bremer Ratskeller,de,Vor der geführten Besichtigung des Rathauses h...,6,d6c3730a-097d-465d-b48c-d9e0b35e5d07,de
4,397,886089135,10322027,14,Super Ambiente und noch besserer Service!,2023-04-13,Ruksson,"Celle, Deutschland","Ich hatte das Vergnügen, im LOC zu speisen und...",5.0,hamburg,Ola Lisboa,de,"Ich hatte das Vergnügen, im Ola Lisboa zu spei...",0,86029d68-a742-4708-906c-22b09557e709,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,921,883130438,6847690,0,Massivst überbewertet,2023-03-22,Comillano,"Frankfurt am Main, Deutschland",Totale Abzocke und massivst überbewertet.,1.0,frankfurt,Thai & Turf - Steakhouse,de,Totale Abzocke und massivst überbewertet. Das ...,0,f6a8f9aa-3ec0-4090-86c4-4629d06018b3,de
3996,603,874144809,947503,2,"Hier geht man nur rein, wenn man woanders kein...",2023-01-06,Bernhard R,Erding,Schönes bairisches Wirtshaus mit katastrophale...,2.0,münchen,Augustiner Stammhaus,de,Schönes bairisches Wirtshaus mit katastrophale...,0,3933eb71-5b09-4edc-aa96-5323ae42370a,de
3997,2009,865055047,967507,1,Mehr Schein als Sein!,2022-10-18,reinhold0913,NaN,Die Sauce und das Gemüse waren in Ordnung.,3.0,dresden,Restaurant & Grand Café Coselpalais,de,Wir waren zum Abendessen im Coselpalais. Ein s...,6,b12cb643-9ef0-494b-adf8-713833595562,de
3998,1654,895902732,1343851,0,Solide und ehrlich bei akzeptablen Preisen,2023-06-19,volkers542,"Sindelfingen, Deutschland",Jahrhundertealte Weinfassböden mit tollen Schn...,4.0,bremen,Bremer Ratskeller,de,Vor der geführten Besichtigung des Rathauses h...,7,08638290-6f50-4440-b386-aae8b8a9601c,de


In [21]:
df_balanced_reviews_sentences["rating"].value_counts()

rating
1.0    800
2.0    800
3.0    800
4.0    800
5.0    800
Name: count, dtype: int64